In [ ]:
from lib.utils import *
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader,TensorDataset
import torch
from torch import nn
from lib.env import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lib.utils import UniformRandomClassifier,ProportionalRandomClassifier

In [ ]:
eval = []

In [ ]:
for (train_ids,test_id) in tqdm(get_leave_one_out_cv_ids_for_ekyn()):
    X_train,y_train = get_trainloader_windowed(train_ids)

    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)

    X_test,y_test = get_testloader_windowed(test_id)

    y_pred = clf.predict(X_test)
    cm_grid(y_test.argmax(axis=1),y_pred.argmax(axis=1))
    eval.append(metrics(y_test,y_pred))

In [ ]:
df = pd.DataFrame([[fold['precision'] for fold in np.load('proportional_random_eval_unscaled.npy',allow_pickle=True)],
    [fold['recall'] for fold in np.load('proportional_random_eval_unscaled.npy',allow_pickle=True)],
    [fold['f1'] for fold in np.load('proportional_random_eval_unscaled.npy',allow_pickle=True)],['proportional']*16],index=['precision','recall','f1','classifier']).T
df = pd.concat([df,pd.DataFrame([[fold['precision'] for fold in np.load('uniform_random_eval_unscaled.npy',allow_pickle=True)],
    [fold['recall'] for fold in np.load('uniform_random_eval_unscaled.npy',allow_pickle=True)],
    [fold['f1'] for fold in np.load('uniform_random_eval_unscaled.npy',allow_pickle=True)],['uniform']*16],index=['precision','recall','f1','classifier']).T])
df = pd.concat([df,pd.DataFrame([[fold['precision'] for fold in np.load('mlp_eval_unscaled.npy',allow_pickle=True)],
    [fold['recall'] for fold in np.load('mlp_eval_unscaled.npy',allow_pickle=True)],
    [fold['f1'] for fold in np.load('mlp_eval_unscaled.npy',allow_pickle=True)],['mlp']*16],index=['precision','recall','f1','classifier']).T])
df = pd.concat([df,pd.DataFrame([[fold['precision'] for fold in np.load('rf_eval_unscaled.npy',allow_pickle=True)],
    [fold['recall'] for fold in np.load('rf_eval_unscaled.npy',allow_pickle=True)],
    [fold['f1'] for fold in np.load('rf_eval_unscaled.npy',allow_pickle=True)],['rf']*16],index=['precision','recall','f1','classifier']).T])
df[['precision','recall','f1']] = df[['precision','recall','f1']].astype(float)
df['classifier'] = df['classifier'].astype(str)
df = df.reset_index(drop=True)
sns.violinplot(data=df,x='f1',y='classifier',split=True)

In [ ]:
train_ids,test_id = get_leave_one_out_cv_ids_for_ekyn()[0]
X_train,y_train = get_trainloader_windowed(train_ids)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_dev,y_train,y_dev = train_test_split(X_train,y_train,test_size=.1,shuffle=True,random_state=0)

In [ ]:
class CustomMLP(nn.Module):
    def __init__(self,input_size=210) -> None:
        super().__init__()
        self.fc1 = nn.Linear(input_size,512)
        self.d1 = nn.Dropout1d(p=.1)

        self.fc2 = nn.Linear(512,512)
        self.d2 = nn.Dropout1d(p=.2)

        self.fc3 = nn.Linear(512,3)


    def forward(self,x):
        x = self.fc1(x)
        x = relu(x)
        x = self.d1(x)

        x = self.fc2(x)
        x = relu(x)
        x = self.d2(x)

        x = self.fc3(x)
        return x

In [ ]:
from lib.models import MLP
model = MLP(input_size=210).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=32,shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=32,shuffle=True)

In [ ]:
lossi = []
loss_tr = []

model.train()
loss_dev = []

for i in range(100):
    loss_tr_total = 0

    for (X_tr,y_tr) in tqdm(trainloader):
        X_tr,y_tr = X_tr.to(DEVICE),y_tr.to(DEVICE)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi.append(loss.item())
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))

    with torch.no_grad():
        loss_dev_total = 0
        for (X_dv,y_dv) in devloader:
            X_dv,y_dv = X_dv.to(DEVICE),y_dv.to(DEVICE)
            logits = model(X_dv)
            loss = criterion(logits,y_dv)
            loss_dev_total += loss.item()
        loss_dev.append(loss_dev_total/len(devloader))

In [ ]:
plt.plot(loss_tr)
plt.plot(loss_dev)
# .62 with MLP


In [ ]:
loss,metrics,y_true,y_pred,y_logits = evaluate(trainloader,model,criterion)
cm_grid(y_true,y_pred)

In [ ]:
X_test,y_test = get_testloader_windowed(test_id)
testloader = DataLoader(TensorDataset(X_test,y_test),batch_size=32,shuffle=True)

loss,metrics,y_true,y_pred,y_logits = evaluate(trainloader,model,criterion)
cm_grid(y_true,y_pred)